In [3]:
import pandas as pd

# 1️⃣ 기존 전처리 완료 데이터 불러오기 (이진라벨 포함 데이터 기준)
df = pd.read_csv("p24_final_labeled_binary.csv", encoding='utf-8-sig')
df = df.dropna(subset=['소외이진라벨'])

# 2️⃣ 파생변수 생성

# ✅ 디지털 적극성 지수 (Active Score)
df['Active_Score'] = (
    df['SNS피드확인'] + 
    df['SNS좋아요댓글'] + 
    df['OTT이용빈도']
)

# ✅ 정보판별 종합지수 (Information Literacy Index)
df['Info_Literacy_Index'] = (
    df['정보판별1(사실확인)'] + 
    df['정보판별2(근거확인)'] + 
    df['정보판별3(상업적의도확인)'] + 
    df['정보판별4(정치적의도확인)']
)

# ✅ SNS 소통강도 (SNS Intensity)
df['SNS_Intensity'] = (
    df['SNS피드확인'] + 
    df['SNS좋아요댓글']
)

# ✅ 고령 여부 (고령군 이진화)
df['고령여부'] = df['연령'].apply(lambda x: 1 if x >= 6 else 0)

# 3️⃣ 결과 확인
print(df.head())

# 4️⃣ 파생변수 포함 최종 데이터 저장
df.to_csv("p24_final_labeled_with_features.csv", index=False, encoding='utf-8-sig')


   성별  연령  학력  직업  소득  스마트폰보유  검색능력  정보판별1(사실확인)  정보판별2(근거확인)  정보판별3(상업적의도확인)  \
0   1   6   5   0   1       1     5            2            2               3   
1   1   3   5   1   5       1     5            2            2               3   
2   0   3   5   1   5       1     5            2            2               1   
3   0   6   5   1   5       1     5            2            2               2   
4   0   4   5   1   5       1     5            3            2               3   

   ...  OTT이용빈도  AI인지수준  AI이용여부  소득그룹  소외군라벨  소외이진라벨  Active_Score  \
0  ...      8.0       2       0     0      1       1          23.0   
1  ...      8.0       2       0     1      1       1          25.0   
2  ...      8.0       3       0     1      2       0          21.0   
3  ...      8.0       3       0     1      2       0          24.0   
4  ...      8.0       3       0     1      2       0          23.0   

   Info_Literacy_Index  SNS_Intensity  고령여부  
0                   10           15.0     1  


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

# 1️⃣ 데이터 불러오기 (파생변수 포함 최종 데이터셋)
df = pd.read_csv("p24_final_labeled_with_features.csv", encoding='utf-8-sig')
df = df.dropna(subset=['소외이진라벨'])

# 2️⃣ X, y 설정 (파생변수까지 포함)
X = df[[
    '검색능력', '정보판별1(사실확인)', '정보판별2(근거확인)',
    '정보판별3(상업적의도확인)', '정보판별4(정치적의도확인)',
    'SNS피드확인', 'SNS좋아요댓글', 'OTT이용빈도',
     '소득그룹', '연령',
    'Active_Score', 'Info_Literacy_Index', 'SNS_Intensity', '고령여부'
]]
y = df['소외이진라벨']

# 3️⃣ Train-Test 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4️⃣ SMOTE 적용 (결측 제거 후 진행)
X_train_no_na = X_train.dropna()
y_train_no_na = y_train.loc[X_train_no_na.index]

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_no_na, y_train_no_na)

# 5️⃣ 테스트셋 결측 제거 (공정 비교)
X_test_no_na = X_test.dropna()
y_test_no_na = y_test.loc[X_test_no_na.index]

# 6️⃣ RandomForest (튜닝 반영)
rf_model = RandomForestClassifier(
    n_estimators=300, max_depth=None, min_samples_split=10,
    class_weight='balanced', random_state=42
)
rf_model.fit(X_train_resampled, y_train_resampled)
y_pred_rf = rf_model.predict(X_test_no_na)
print("=== Random Forest (튜닝, 파생포함) ===")
print(classification_report(y_test_no_na, y_pred_rf))

# 7️⃣ Logistic Regression
log_model = LogisticRegression(max_iter=1000, class_weight='balanced')
log_model.fit(X_train_resampled, y_train_resampled)
y_pred_log = log_model.predict(X_test_no_na)
print("=== Logistic Regression (파생포함) ===")
print(classification_report(y_test_no_na, y_pred_log))

# 8️⃣ XGBoost
xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train_resampled, y_train_resampled)
y_pred_xgb = xgb_model.predict(X_test_no_na)
print("=== XGBoost (파생포함) ===")
print(classification_report(y_test_no_na, y_pred_xgb))


=== Random Forest (튜닝, 파생포함) ===
              precision    recall  f1-score   support

           0       0.63      0.69      0.66       452
           1       0.69      0.63      0.66       504

    accuracy                           0.66       956
   macro avg       0.66      0.66      0.66       956
weighted avg       0.66      0.66      0.66       956

=== Logistic Regression (파생포함) ===
              precision    recall  f1-score   support

           0       0.60      0.69      0.64       452
           1       0.68      0.59      0.63       504

    accuracy                           0.64       956
   macro avg       0.64      0.64      0.64       956
weighted avg       0.64      0.64      0.64       956

=== XGBoost (파생포함) ===
              precision    recall  f1-score   support

           0       0.63      0.70      0.66       452
           1       0.70      0.64      0.67       504

    accuracy                           0.67       956
   macro avg       0.67      0.67    

C:\Users\windo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [14:24:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [ ]:
#파생변수를 추가 생성하여 모델의 해석력을 보강하였다. 다만 기본 행동변수들의 설명력이 이미 상당히 높아, 파생변수 추가에 따른 성능 개선은 제한적으로 나타났다.
#이미 기존 입력변수들이 상당히 설명력이 높은 구조였다.
# 파생변수가 '의미적으로 좋은 변수'는 맞지만, 예측력 향상에는 한계